In [1]:
#%load_ext nb_black

In [2]:
import os
import boto3
import logging
from datetime import datetime

boto3.set_stream_logger(name="botocore.credentials", level=logging.WARNING)

In [3]:
import sagemaker
from sagemaker.transformer import Transformer

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\pat\AppData\Local\sagemaker\sagemaker\config.yaml


In [6]:
sgmkr_clnt = boto3.client("sagemaker")
sgmkr_rt = boto3.client("runtime.sagemaker")

In [7]:
# role_arn = sagemaker.get_execution_role()
#role_arn = os.getenv("SGMKR_ROLE_ARN")
role_arn = "arn:aws:iam::012345678901:role/service-role/AmazonSageMaker-ExecutionRole-20240718T104942"

#### Create model

In [8]:
from sagemaker import image_uris

In [12]:
region = sagemaker.Session().boto_region_name
print(region)
model_img = sagemaker.image_uris.retrieve("xgboost", region, "latest")
print(model_img)

us-east-1
811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest


In [13]:
#model_img = "811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest"

In [23]:
!aws s3 ls pat-demo-bkt

                           PRE iris/


In [24]:
bucket = 'pat-demo-bkt'
prefix = 'iris/model/iris-xgboost-2024-10-17-17-03-48/output'
model_tar_file = "model.tar.gz"
s3_model_path = "s3://{}/{}/{}".format(bucket, prefix, model_tar_file)
s3_model_path_ls = "{}/{}/{}".format(bucket, prefix, model_tar_file)
#s3_model_path = "s3://pat-demo-bkt/iris/model/iris-xgboost-2024-10-17-17-03-48/output/model.tar.gz"
print (s3_model_path_ls)
!aws s3 ls {s3_model_path_ls}

pat-demo-bkt/iris/model/iris-xgboost-2024-10-17-17-03-48/output/model.tar.gz
2024-10-17 17:07:05       3453 model.tar.gz


In [25]:
model_path = (
    s3_model_path
)
print (model_path)

s3://pat-demo-bkt/iris/model/iris-xgboost-2024-10-17-17-03-48/output/model.tar.gz


In [26]:
model_name = "iris-xgboost-" + datetime.today().strftime("%Y-%m-%d-%H-%M-%S")
print(model_name)

iris-xgboost-2024-10-18-13-08-20


In [27]:
response = sgmkr_clnt.create_model(
    ModelName=model_name,
    PrimaryContainer={"Image": model_img, "ModelDataUrl": model_path},
    ExecutionRoleArn=role_arn,
)

print(response)

{'ModelArn': 'arn:aws:sagemaker:us-east-1:012345678901:model/iris-xgboost-2024-10-18-13-08-20', 'ResponseMetadata': {'RequestId': '18cf1c5b-d894-40e6-a379-a6a4b5c6d94c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '18cf1c5b-d894-40e6-a379-a6a4b5c6d94c', 'content-type': 'application/x-amz-json-1.1', 'content-length': '94', 'date': 'Fri, 18 Oct 2024 20:08:29 GMT'}, 'RetryAttempts': 0}}


In [28]:
bucket = "pat-demo-bkt"
prefix_bt = "iris"

#### Batch Transform

In [29]:
batch_ip = 's3://{}/{}/{}'.format(bucket, prefix_bt, 'batch_transform')
batch_op = 's3://{}/{}/{}'.format(bucket, prefix_bt, 'batch_transform')

In [30]:
transformer = Transformer(
    model_name=model_name,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    output_path=batch_op,
)

transformer.transform(
    data=batch_ip, data_type="S3Prefix", content_type="text/csv"
)
transformer.wait()

INFO:sagemaker:Creating transform job with name: xgboost-2024-10-18-20-13-17-568


.................................
Arguments: serve
[2024-10-18 20:19:22 +0000] [1] [INFO] Starting gunicorn 19.9.0
[2024-10-18 20:19:22 +0000] [1] [INFO] Listening at: http://0.0.0.0:8080 (1)
[2024-10-18 20:19:22 +0000] [1] [INFO] Using worker: gevent
[2024-10-18 20:19:22 +0000] [21] [INFO] Booting worker with pid: 21
[2024-10-18 20:19:22 +0000] [22] [INFO] Booting worker with pid: 22
[2024-10-18 20:19:22 +0000] [23] [INFO] Booting worker with pid: 23
/opt/amazon/lib/python3.7/site-packages/gunicorn/workers/ggevent.py:65: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.util (/opt/amazon/lib/python3.7/site-packages/urllib3/util/__init__.py)', 'urllib3.util.ssl_ (/opt/amazon/lib/python3.7/site-packages/ur

#### Endpoint

In [31]:
ep_config_name = "tmp-ep-config-" + datetime.today().strftime("%Y-%m-%d-%H-%M-%S-%f")
print(ep_config_name)

tmp-ep-config-2024-10-18-14-16-13-624785


In [32]:
response = sgmkr_clnt.create_endpoint_config(
    EndpointConfigName=ep_config_name,
    ProductionVariants=[
        {
            "VariantName": "version-1",
            "ModelName": model_name,
            "InitialInstanceCount": 1,
            "InstanceType": "ml.m4.xlarge",
            # sever_less = ''
        },
    ],
)

print(response)

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:012345678901:endpoint-config/tmp-ep-config-2024-10-18-14-16-13-624785', 'ResponseMetadata': {'RequestId': '3a356224-e954-4d23-9fef-cd5173188682', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '3a356224-e954-4d23-9fef-cd5173188682', 'content-type': 'application/x-amz-json-1.1', 'content-length': '121', 'date': 'Fri, 18 Oct 2024 21:16:58 GMT'}, 'RetryAttempts': 0}}


In [33]:
ep_name = "tmp-ep-" + datetime.today().strftime("%Y-%m-%d-%H-%M-%S-%f")
print(ep_name)

tmp-ep-2024-10-18-14-19-04-897502


In [34]:
response = sgmkr_clnt.create_endpoint(
    EndpointName=ep_name, EndpointConfigName=ep_config_name,
)
print(response)

{'EndpointArn': 'arn:aws:sagemaker:us-east-1:012345678901:endpoint/tmp-ep-2024-10-18-14-19-04-897502', 'ResponseMetadata': {'RequestId': 'ce49bffd-0da9-492d-a6a8-15765862c0ee', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'ce49bffd-0da9-492d-a6a8-15765862c0ee', 'content-type': 'application/x-amz-json-1.1', 'content-length': '101', 'date': 'Fri, 18 Oct 2024 21:19:12 GMT'}, 'RetryAttempts': 0}}


In [35]:
waiter = sgmkr_clnt.get_waiter("endpoint_in_service")
waiter.wait(EndpointName=ep_name, WaiterConfig={"Delay": 123, "MaxAttempts": 123})
print("Endpoint created")

Endpoint created


In [36]:
payload = "7.7, 3.0, 6.1, 2.3"
# payload = '7.7, 3.0, 6.1, 2.3 \n 7.9, 3.8, 6.4, 2.1'

In [37]:
sgmkr_runt = boto3.client("runtime.sagemaker")

In [38]:
response = sgmkr_runt.invoke_endpoint(
    EndpointName=ep_name, ContentType="text/csv", Body=payload,
)

prediction = response["Body"].read().decode()
print(prediction)

2.0


In [40]:
print("Deleting sagemaker endpoint")
response = sgmkr_clnt.delete_endpoint(EndpointName = ep_name)  
print("Deleted sagemaker endpoint")
#print(response)    

Deleting sagemaker endpoint
Deleted sagemaker endpoint


In [41]:
print("Deleting sagemaker endpoint configuration")
response = sgmkr_clnt.delete_endpoint_config(EndpointConfigName = ep_config_name)
print("Deleted sagemaker endpoint configuration")
#print(response)

Deleting sagemaker endpoint configuration
Deleted sagemaker endpoint configuration
